In [12]:
import os

import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()


def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(os.getenv('DATABASE'))
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

def load_features(select) -> pd.DataFrame:
    return batch_load_sql(select)


In [14]:
query = """
SELECT *
FROM post_text_df
"""
posts_df = load_features(query)

posts_df.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [15]:
query = """
SELECT COUNT(*), post_id
FROM feed_data
WHERE action = 'like'
GROUP BY post_id;
"""
posts_likes = load_features(query)
posts_likes = posts_likes.set_index('post_id')

In [16]:
posts_df['total_likes'] = posts_df['post_id'].map(posts_likes['count'])

In [17]:
query = """
SELECT COUNT(*), post_id
FROM feed_data
GROUP BY post_id;
"""
posts_actions = load_features(query)
posts_actions = posts_actions.set_index('post_id')

In [18]:
posts_df['like_rate'] = posts_df['post_id'].map(posts_likes['count']/posts_actions['count'])

In [19]:
posts_df.head()

,post_id,text,topic,total_likes,like_rate
0,1,UK economy facing major risks\n\nThe UK manufa...,business,1067.0,0.111436
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,637.0,0.078333
2,3,Asian quake hits European shares\n\nShares in ...,business,1122.0,0.117684
3,4,India power shares jump on debut\n\nShares in ...,business,1171.0,0.125523
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,1153.0,0.118426
